# Groupby and Arrest Data

In our merging exercises, we examined the relationship between county-level violent arrest totals and county-level drug arrest totals. In those exercises, you were given a dataset that provided you with county-level arrest totals. But that's not actually how the data is provided by the state of California. This week we will work with the *raw* California arrest data, which is not organized by county or even county-year. 

## Gradescope Autograding

This assignment will be autograded using the same autograder we have used for our past several exercises. The only requirements for submitting this assignment are:

- you may only import pandas, numpy, matplotlib, and altair.
- you import data **from a URL** (since the autograder in the cloud won't see your file system)
- you name your notebook file `exercise_groupby.ipynb`
- you store your solutions in a dictionary called `results` with answers assigned to the keys provided in the prompts below.
- your notebook runs from the start to completion without any errors. That means that for questions that (deliberately) invite you to write code that *causes* errors, before submitting comment out the code that generates the error to ensure it won't interrupt notebook execution.

Your notebook will be executed by the autograder and the value of identified dictionary entries will then be checked against solution values. However, be aware TAs will _also_ review your notebooks for answers to other questions and to ensure your notebook is well written and formatted.

You can check that you have answers for all questions in your `results` dictionary with this code:

```python
assert set(results.keys()) == {
    "ex4_num_rows",
    "ex5_collapsed_vars",
    "ex7_alameda_1980_share_violent_arrestees_black",
    "ex11_white_drug_share",
    "ex11_black_drug_share",
    "ex12_proportionate",
}

```

### Submission Limits

Please remember that you are **only allowed four submissions to the autograder.** Your last submission (if you submit 4 or fewer times), or your third submission (if you submit more than 4 times) will determine your grade. 



### Exercise 1

Import the raw California arrest data from the State Attorney General's office [here](https://openjustice.doj.ca.gov/data) by scrolling down to the "Arrests" category and copying the link for the "Arrests - CSV, 6.1 MB" file (remember: you must import your data using a link for the autograder to be able to assess your work).

Note that if you were getting a `SSLError`, you can try reading a [copy of the data here.](https://github.com/nickeubank/MIDS_Data/blob/master/OnlineArrestData1980-2021.csv)

## Learning the Group Structure of Your Data

### Exercise 2

What is the unit of observation for this dataset? In other words, when row zero says that there were 505 arrests for `VIOLENT` crimes, what exactly is that telling you -- 505 arrests in 1980? 505 arrests in Alameda County?

### Testing Your Assumptions

It's important to be able to test whether the data you are working with really is organized the way you think it is, especially when working with groupby, so let's discuss how to check your answer to number 2 with `duplicated`. Consider the following data:

In [2]:
import pandas as pd
import numpy as np
df = pd.DataFrame(
    {
        "social_security_numbers": [
            111111111,
            222222222,
            222222222,
            333333333,
            333333333,
        ],
        "second_column": ["a", "a", "a", "a", "b"],
    }
)
df

,social_security_numbers,second_column
0,111111111,a
1,222222222,a
2,222222222,a
3,333333333,a
4,333333333,b


If we want to see if there are any duplicate rows in the dataset, we can use `.duplicated()`:

In [8]:
df.duplicated()


0    False
1    False
2     True
3    False
4    False
dtype: bool

As you can see, `.duplicated()` looks at each row, and returns `True` if it has seen the row it is looking at before. Note that it doesn't tag *all* the rows that look similar -- it treats the first instance of a row as unique, and only tags subsequent repitions are "duplicates" (You can change this behavior with keyword arguments if you want all rows tagged).

Duplicated can also be used to test for duplicates on a sub-set of rows. For example, if we want to test for rows with duplicate values of the variable `social_security_numbers`, we can type:

In [9]:
df.duplicated(["social_security_numbers"])


0    False
1    False
2     True
3    False
4     True
dtype: bool

Since `duplicated` is now only looking at the first column, the last row is now a duplicate (because 333333333 is duplicated), where when we considered all columns, it was not a duplicate (because the value in the second column varied. 

We can now pair `.duplicated()` with the `.any()` function to test for the presence of duplicates in your dataset, which is how we test if we really understand what constitutes a unique observation (i.e. if we think each row of our data is a unique person, then we shouldn't see any duplicated values of social security numbers, which are unique to each person in the United States). 

When you run `.any()` on an array of booleans, it returns a single value of `True` if *any* entries are `True`, and a single value of `False` if *no* entries are `True`. (You can also use `.all()` to test if all entries are false). 

Thus the command: `df.duplicated(['social_security_numbers'])` will return `False` if `social_security_numbers` uniquely idenfies every row in our dataset (since there are no duplicates)! If any rows are duplicated, then `social_security_numbers` doesn't uniquely identify our observations (i.e. each row does not represent a unique person):

In [10]:
df.duplicated(["social_security_numbers"]).any()


True

This might feel backward, so you can also add a `not` before the test if you want. :) In fact, in my code I add an explicit test using the `assert` statement. The command `assert` says "if the thing that follows this is `True`, don't do anything; if it's False, raise an exception. So in my code, I often write:

```python
assert not df.duplicated(["social_security_numbers"]).any()

---------------------------------------------------------------------------
AssertionError                            Traceback (most recent call last)
Cell In [20], line 1
----> 1 assert not df.duplicated(["social_security_numbers"]).any()

AssertionError: 
```

(which in this case raises an exception! Because the rows *aren't* unique!)

### Exercise 3

Use `duplicated` to test if the variables *you* think uniquely identify rows in your data really do uniquely identify rows. If you were wrong, update your beliefs (and your code) until you have an `assert` statement that passes.

### Exercise 4

Once you have a handle on how the data looks now, please **collapse the data** to be one observation per county-year-racial group. Store the number of rows in the resulting dataframe in `ex4_num_rows`.

**Hint:** Think carefully about the most appropriate *aggregation function* (e.g., mean, median, sum, count) given the data we're working with!

**Hint 2:** When using `groupby`, I am a HUGE fan of the `as_index=False` keyword argument (e.g., `df.groupby("col1", as_index=False).mean()`). By default when you use groupby, pandas likes to make a new index out of the grouping columns. As you know, I HATE indices, and this keyword argument prevents pandas from this behavior. To illustrate:

In [12]:
df = pd.DataFrame(
    {
        "social_security_numbers": [
            111111111,
            222222222,
            222222222,
            222222222,
            333333333,
        ],
        "second_column": ["a", "a", "b", "a", "b"],
        "third_column": [100, 200, 300, 400, 500],
    }
)
df


,social_security_numbers,second_column,third_column
0,111111111,a,100
1,222222222,a,200
2,222222222,b,300
3,222222222,a,400
4,333333333,b,500


In [13]:
# Without as_index=False
df.groupby(["social_security_numbers", "second_column"]).mean()

third_column
social_security_numbers second_column              
111111111               a                     100.0
222222222               a                     300.0
                        b                     300.0
333333333               b                     500.0

In [14]:
# With as_index=False
df.groupby(["social_security_numbers", "second_column"], as_index=False).mean()

,social_security_numbers,second_column,third_column
0,111111111,a,100.0
1,222222222,a,300.0
2,222222222,b,300.0
3,333333333,b,500.0


In [1]:
import numpy as np

df["avg"] = df.groupby(["social_security_numbers", "second_column"], as_index=False)[
    "third_column"
].transform(np.mean)

NameError: name 'df' is not defined

**BUT** Don't use `as_index=False` when using `.transform()`—starting in pandas 1.5, if you use `as_index=False` and try and insert the values back into the original dataframe, you'll get an error:

```python
df["avg"] = df.groupby(["social_security_numbers", "second_column"], as_index=False)["third_column"].transform(np.mean)


Only use this trick when NOT using `.transform()`.

### Exercise 5

You should notice that there are significantly fewer rows in the data now. Given your answer from 3, what groups were you *collapsing* in question 4 (in other words, which different groups were being aggregated over)?

Store the names of those variables **in a `list` in alphabetical order** in `results` as `"ex5_collapsed_vars"`.

## Racial Arrest Composition

The next question we want to ask is: does the racial composition of arrests in each county vary by arrest type? In other words, do Blacks, Hispanics, Whites, and Other make up substantially different portions of the people arrested for drug offenses and violent offenses?

To answer this question, we need to be able to say what proportion of all drug arrestees were Black/White/Hispanic/Other and what proportion of all violent arrestees were Black/White/Hispanic/Other *for each county and for each year*.

In trying to do this, we'll need to break the problem down into pieces, starting from our goal (these quantities) and working backwards.

### Exercise 6

To calculate the share of all drug and violent arrestees who come from each racial group, what quantities do we need? 

Expressed differently, what two variables (one for drug arrests, one for violent arrests) do we need to add to our data to it allow us to calculate the share of arrestees in each county-year that come from a given racial group?

Calculate these two variables (one variable for drug arrests, one for violent arrests). 

**Hint:** `transform` should probably make an appearance here.

**Hint:** This types of groupby manipulations are an easy place to do things wrong, so it's important to eye-ball your data to be sure you did things right! 

Take a look at the data for, say, Alameda County in 1980, and compare the values that you calculate above with what you get if you try to calculate those same numbers by hand using the raw data from Exercise 5?

### Exercise 7

Now calculate the share of violent and drug arrestees in each county and year from each racial group.

To ensure you're on the right track, store the share (between zero and one) of violent arrestees in Alameda County in 1980 who were Black as `ex7_alameda_1980_share_violent_arrestees_black`.

### Exercise 8

We're about to start studying this data by plotting the share of violent arrestees that are Black against the share of felony drug arrestees that are Black. But the moment where you finish your data manipulations and are about to start you data analysis is a *great* time to just make sure everything in your data looks good. Let's run a few checks:

- Are your values of the share of felony arrestees who were arrested for violent crimes sensible?
- You're about to analyze the data using only the rows for the Black racial group. How many unique counties are there with data for Black arrestees? How many for White arrestees? Do you remember how many counties there are in CA (google is your friend if not!)?

Include assert statements to check all these features of your data.

(Reminder: the autograder will only work if you are entire notebook runs without errors.)

If you encounter any problems, make sure you understand why they are happening, and fix them (if a fix is appropriate!).

### Exercise 9

Now check to see if you have the same number of counties *in each year for each racial group*.

Can you figure out why there aren't? (Hint: this isn't trivial to figure out! You'll probably have to do some detective work, and even then you'll have to make a bit of a "best guess" at what's going on!)

### Exercise 10

Once you think you have a theory of the case, fix the problem. Again, this will NOT be trivial.

### Exercise 11

Now that you have corrected your problem, averaging over the values for each county-year, calculate the average share of drug arrestees who are Black and the average share of drug arrestees who are White. Store the results as `"ex11_white_drug_share"` and `"ex11_black_drug_share"`.

### Exercise 12

Plot the share of violent arrestees that are Black against the share of felony drug arrestees that are Black. Do they look proportionate? Answer in text here.

Do Black arrestees make up a greater share of violent arrestees, or a greater share of drug arrestees? Store you answer as `"greater violent"`, or `"greater drug"` in `"ex12_proportionate"`.

**Hint:** You can add a 45 degree reference line by making a dummy dataset with one point at 0,0 and one point at a location near the top of your data (e.g. 0.7, 0.7). Without a reference line it's VERY hard to be sure if how your points relate to proportionality.

(A quick note of warning on interpretation: these results can tell you whether Black Californians make up a larger proportion of *arrests* for certain types of crimes, not whether they make up a larger proportion of people who *commit* a give type of crime! Those *might* be the same, but they might not... this data just can't answer that question.)